In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('Practise').getOrCreate()

In [4]:
spark

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [5]:
df_pyspark=spark.read.option('header','true').csv('C:/Users/Lenovo/Desktop/PySpark_Task_4/SaleData.csv',inferSchema=True)

In [6]:
df_pyspark.show()

+---------+-------+-------+---------+------------+-----+----------+
|OrderDate| Region|Manager| SalesMan|        Item|Units|Unit_price|
+---------+-------+-------+---------+------------+-----+----------+
|   1/6/18|   East| Martha|Alexander|  Television| 95.0|    1198.0|
|  1/23/18|Central|Hermann|   Shelli|Home Theater| 50.0|     500.0|
|   2/9/18|Central|Hermann|     Luis|  Television| 36.0|    1198.0|
|  2/26/18|Central|Timothy|    David|  Cell Phone| 27.0|     225.0|
|  3/15/18|   West|Timothy|  Stephen|  Television| 56.0|    1198.0|
|   4/1/18|   East| Martha|Alexander|Home Theater| 60.0|     500.0|
|  4/18/18|Central|   NULL|   Steven|  Television| 75.0|    1198.0|
|   5/5/18|Central|Hermann|     Luis|  Television| 90.0|    1198.0|
|  5/22/18|   West|Douglas|  Michael|  Television| 32.0|    1198.0|
|   6/8/18|   East| Martha|Alexander|Home Theater| 60.0|     500.0|
|  6/25/18|Central|Hermann|    Sigal|  Television| 90.0|    1198.0|
|  7/12/18|   East| Martha|    Diana|Home Theate

In [7]:
df_pyspark=df_pyspark.na.fill('Others')
df_pyspark.show()

+---------+-------+-------+---------+------------+-----+----------+
|OrderDate| Region|Manager| SalesMan|        Item|Units|Unit_price|
+---------+-------+-------+---------+------------+-----+----------+
|   1/6/18|   East| Martha|Alexander|  Television| 95.0|    1198.0|
|  1/23/18|Central|Hermann|   Shelli|Home Theater| 50.0|     500.0|
|   2/9/18|Central|Hermann|     Luis|  Television| 36.0|    1198.0|
|  2/26/18|Central|Timothy|    David|  Cell Phone| 27.0|     225.0|
|  3/15/18|   West|Timothy|  Stephen|  Television| 56.0|    1198.0|
|   4/1/18|   East| Martha|Alexander|Home Theater| 60.0|     500.0|
|  4/18/18|Central| Others|   Steven|  Television| 75.0|    1198.0|
|   5/5/18|Central|Hermann|     Luis|  Television| 90.0|    1198.0|
|  5/22/18|   West|Douglas|  Michael|  Television| 32.0|    1198.0|
|   6/8/18|   East| Martha|Alexander|Home Theater| 60.0|     500.0|
|  6/25/18|Central|Hermann|    Sigal|  Television| 90.0|    1198.0|
|  7/12/18|   East| Martha|    Diana|Home Theate

In [8]:
original_count = df_pyspark.count()
# Drop duplicates
df_pyspark = df_pyspark.dropDuplicates()
deduplicated_count = df_pyspark.count()
duplicates_present = original_count != deduplicated_count
if duplicates_present:
    print("Duplicates were present.Removed Duplicates")
else:
    print("No duplicates found.")
df_pyspark.show()

Duplicates were present.Removed Duplicates
+---------+-------+-------+---------+------------+-----+----------+
|OrderDate| Region|Manager| SalesMan|        Item|Units|Unit_price|
+---------+-------+-------+---------+------------+-----+----------+
| 12/29/18|   East|Douglas|    Karen| Video Games| 74.0|      58.5|
| 12/12/18|Central|Douglas|     John|  Television| 67.0|    1198.0|
|  6/17/19|Central|Hermann|   Shelli|        Desk|  5.0|     125.0|
|   9/1/18|Central|Douglas|     John|        Desk|  2.0|     125.0|
|   4/1/18|   East| Martha|Alexander|Home Theater| 60.0|     500.0|
|   2/9/18|Central|Hermann|     Luis|  Television| 36.0|    1198.0|
|   8/7/19|Central|Hermann|   Others| Video Games| 42.0|      58.5|
|  7/12/18|   East| Martha|    Diana|Home Theater| 29.0|     500.0|
|   7/4/19|   East| Martha|Alexander| Video Games| 62.0|      58.5|
|  4/27/19|   East| Martha|    Diana|  Cell Phone| 96.0|     225.0|
|  9/18/18|   East| Martha|   Others| Video Games| 16.0|      58.5|
| 11/

In [9]:
from pyspark.sql.functions import col, sum
df_with_sales_amount = df_pyspark.withColumn("Total_sales_amount", col("Units") * col("Unit_price"))
total_sales_per_product = df_with_sales_amount.groupBy("Item") \
    .agg(sum("Total_sales_amount").alias("Total_sales_amount"))

# Show the result
total_sales_per_product.show()

+------------+------------------+
|        Item|Total_sales_amount|
+------------+------------------+
|        Desk|            1250.0|
| Video Games|           23107.5|
|  Television|          857768.0|
|  Cell Phone|           62550.0|
|Home Theater|          404500.0|
+------------+------------------+



In [ ]:
# Output the results to a new CSV file
output_file_path = "total_sales_per_product.csv"
total_sales_per_product.write.csv(output_file_path, mode="overwrite", header=True)